In [1]:
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
import pickle as pkl
from keras.optimizers import SGD
import keras.backend as K
from densenet161 import DenseNet 
import numpy as np 
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.models import load_model
import cv2
import numpy as np
from keras.optimizers import SGD
import keras.backend as K
from keras.models import Model
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from imutils import paths
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import pickle as pkl
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import TensorBoard
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)  
KTF.set_session(sess)

Using TensorFlow backend.


In [2]:
nb_class=29

In [3]:
def SaltAndPepper(src,percetage=0.1):#椒盐噪声 越小噪声越少  
    SP_NoiseImg=src.copy()
    SP_NoiseNum=int(percetage*src.shape[0]*src.shape[1]) 
    for i in range(SP_NoiseNum): 
        randR=np.random.randint(0,src.shape[0]-1) 
        randG=np.random.randint(0,src.shape[1]-1) 
        randB=np.random.randint(0,3)
        if np.random.randint(0,1)==0: 
            SP_NoiseImg[randR,randG,randB]=0 
        else: 
            SP_NoiseImg[randR,randG,randB]=255 
    return SP_NoiseImg 
def addGaussianNoise(image,percetage=0.1): #Gauss 噪声 越小噪声越少
    G_Noiseimg = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    G_NoiseNum=int(percetage*image.shape[0]*image.shape[1]) 
    for i in range(G_NoiseNum): 
        temp_x = np.random.randint(0,h) 
        temp_y = np.random.randint(0,w) 
        G_Noiseimg[temp_x][temp_y][np.random.randint(3)] = np.random.randn(1)[0] 
    return G_Noiseimg
#dimming
def darker(image,percetage=0.5):#调节-->暗 范围0~1 参考 min 0.5
    image_copy = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    #get darker
    for xi in range(0,w):
        for xj in range(0,h):
            image_copy[xj,xi,0] = int(image[xj,xi,0]*percetage)
            image_copy[xj,xi,1] = int(image[xj,xi,1]*percetage)
            image_copy[xj,xi,2] = int(image[xj,xi,2]*percetage)
    return image_copy
def brighter(image, percetage=1.5):#调节-->亮 范围1~无穷 参考max 1.5
    image_copy = image.copy()
    w = image.shape[1]
    h = image.shape[0]
    #get brighter
    for xi in range(0,w):
        for xj in range(0,h):
            image_copy[xj,xi,0] = np.clip(int(image[xj,xi,0]*percetage),a_max=255,a_min=0)
            image_copy[xj,xi,1] = np.clip(int(image[xj,xi,1]*percetage),a_max=255,a_min=0)
            image_copy[xj,xi,2] = np.clip(int(image[xj,xi,2]*percetage),a_max=255,a_min=0)
    return image_copy
def rotate(image, angle=15, scale=0.9):#旋转 图像有缺失部分
    w = image.shape[1]
    h = image.shape[0]
    #rotate matrix
    M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
    #rotate
    image = cv2.warpAffine(image,M,(w,h))
    return image
def Geometric_changes(image,width=224,height=224):#仿射变换 扭曲图像 图像可能有缺失部分
    #返回值为 (width,height)的图像
    pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
    x = random.randint(50,100)
    y = random.randint(200,250)
    z = random.randint(10,50)
    pts2 = np.float32([[z, x], [200, 50], [x, y]])
    M = cv2.getAffineTransform(pts1, pts2)
    image_0 = cv2.warpAffine(image, M, (width, height))
    return image_0
def mean_blur(image,k_size=5):#图像模糊，均值滤波 k_size only int 参考范围1~10
    dist = cv2.blur(image,(k_size,k_size))
    return dist
def media_blur(image,k_size=5):#图像模糊 中值模糊 k_size only int and %2==1 参考范围 1~11 一般用于去除椒盐噪声
    dist = cv2.medianBlur(image,k_size)
    return dist
def gauss_blur(image,k_size=5):#图像模糊  Gauss模糊 k_size only int and %2==1 参考范围 1~11 一般用于去除Gauss噪声
    dist= cv2.GaussianBlur(img, (k_size,k_size), 0)
    return dist
def bilater_blur(image,k_size=1,width=75,height=75):#图像模糊 双边滤波
    #k_size:邻域直径，weight height分别是空间高斯函数标准差，灰度值相似性高斯函数标准差
    #k_size only int 参考范围1~10
    dist = cv2.bilateralFilter(img,k_size,width,height)
    return dist
def random_cut(image,width=224,height=224):#图像随机裁剪 Tensorflow
    dist = tf.random_crop(image,[width,height,3])
    sess = tf.InteractiveSession() 
    dist = cv2.cvtColor(dist.eval(),cv2.COLOR_BGR2RGB)
    sess.close()
    return dist

In [4]:
def pic_Aug(image,rand_seed=1):#随机种子 random seed  种子不变每次运行不变
    #处理前先将图像 resize
    random.seed(rand_seed)
    image=cv2.resize(image,(300,300))
    image=random_cut(image)#随机裁剪
    image=mean_blur(image,random.randint(1,10))
    image=cv2.resize(image,(224,224))
    return image

In [5]:
def get_data(image_path_origin="data/train",image_path_gan="data/gen_class/",label_path="data/Train_label.csv"):
    train_label = pd.read_csv("data/Train_label.csv")
    arr=train_label.values
    image_dict={}
    for i in range(arr.shape[0]):
        tmp_arr=list(map(int,arr[i][1].split(';')))
        for j in range(len(tmp_arr)):
            tmp_arr[j]=tmp_arr[j]-1#label 减一 映射到0-28
        image_dict[arr[i][0]]=tmp_arr# dictionary {path:label}
#         print(image_dict[arr[i][0]])
        # image_dict[arr[i][0]]=int(arr[i][1].split(';')[0])-1
    data=[]
    label=[]
    image_paths_origin = sorted(list(paths.list_images(image_path_origin)))
    for im_path in image_paths_origin:
        image=cv2.imread(im_path)
#         image=pic_Aug(image)
        image=cv2.resize(image,(224,224))
        tmp_image_name=im_path.split('/')[2]
        data.append(image)
        label.append(image_dict[tmp_image_name])
    #step two add gan pic
    output_data = open('data.pkl', 'wb')
    output_label= open('label.pkl','wb')
    pkl.dump(data,output_data)
    pkl.dump(label,output_label)

In [6]:
# get_data()

In [7]:
def load_data(data_path='data.pkl',label_path='label.pkl'):
    pkl_data_file = open('data.pkl', 'rb')
    pkl_label_file=open('label.pkl','rb')
    data = pkl.load(pkl_data_file)
    label=pkl.load(pkl_label_file)
    return data,label

In [8]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
	image_paths = sorted(list(paths.list_images(p_path)))
	data=[]
	image_name=[]
	for im_path in image_paths:
		image=cv2.imread(im_path)
		image=cv2.resize(image,(224,224))
		image = img_to_array(image)
		image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
		image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
		image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
		data.append(image)
		image_name.append(im_path.split('/')[2])
	data = np.array(data, dtype="float32")#transfer to numpy
	model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
	sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
	model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
	out= model.predict(data)
	label=np.argmax(out[0])
	print(type(label))
	print(label)
	print(label.shape)
	predict_label = pd.read_csv("data/submit_example.csv")
	for i in range(predict_label.values.shape[0]):
		find_name=predict_label.values[i][0]
		for j in range(data.shape[0]):
			if find_name==image_name[j]:
				predict_label.iat[i,1]=np.argmax(out[j])+1
	predict_label.to_csv("data/predict.csv",index=False)

In [9]:

def load_new_model(weight_path='imagenet_models/densenet161_weights_tf.h5'):
	base_model = DenseNet(reduction=0.5, classes=1000, weights_path=weight_path)
	# model = Model(input=base_model.input, output=base_model.get_layer('conv5_blk_scale').output)
	x=base_model.get_layer('conv5_blk_scale').output
	x = Activation('relu', name='relu_last')(x)
	x=GlobalAveragePooling2D(name='pool_last')(x)
	x= Dense(nb_class, name='fc_last')(x)
	x=Activation('softmax', name='prob_last')(x)
	model=Model(input=base_model.input,output=x,name='new_densenet')
	for layer in model.layers[:-5]:
	    layer.trainable = True
	return model



In [10]:
def train_data(data,label,model):
    (trainX, testX, trainY, testY) = train_test_split(data,
        label, test_size=0.1, random_state=42)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     datagen = ImageDataGenerator(
#         shear_range=0.2, # 随机错切换角度
#         zoom_range=0.3, # 随机缩放范围
#         featurewise_center=False,  # set input mean to 0 over the dataset
#         samplewise_center=False,  # set each sample mean to 0
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset
#         samplewise_std_normalization=False,  # divide each input by its std
#         zca_whitening=False,  # apply ZCA whitening
#         rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
#         width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=True,  # randomly flip images
#         vertical_flip=True)  # randomly flip images
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    print(trainX.shape[0])
    batchs=10
#     model.fit_generator(datagen.flow(trainX, trainY, batch_size=batchs),steps_per_epoch=trainX.shape[0], 
#                         epochs=20, verbose=1,validation_data=(testX,testY),callbacks=[TensorBoard(log_dir='/log_1')])
    model.fit(trainX,trainY,batch_size=10,epochs=15,verbose=1,validation_data=(testX,testY),callbacks=[TensorBoard(log_dir='/logss')])
    model.save_weights('imagenet_models/new_dense_weight.h5')
    model.save('imagenet_models/new_densenet.h5')
    score = model.evaluate(testX,testY,verbose=0)
    print ('accuracy:{}'.format(score[1]))

In [11]:
def predict_data(p_path="data/test",model_path="imagenet_models/new_dense_weight.h5"):
    image_paths = sorted(list(paths.list_images(p_path)))
    data=[]
    image_name=[]
    for im_path in image_paths:
        image=cv2.imread(im_path)
        image=cv2.resize(image,(224,224))
        image = img_to_array(image)
        image[:,:,0] = (image[:,:,0] - 103.94) * 0.017
        image[:,:,1] = (image[:,:,1] - 116.78) * 0.017
        image[:,:,2] = (image[:,:,2] - 123.68) * 0.017
        data.append(image)
        image_name.append(im_path.split('/')[2])
    data = np.array(data, dtype="float32")#transfer to numpy
    model = DenseNet(reduction=0.5, classes=nb_class, weights_path=model_path)
    #origin ls is 1e-2
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    out= model.predict(data)
    return out

In [12]:
data,label=load_data()

In [13]:
data_out=np.zeros((len(data),224,224,3),dtype="float32")

In [14]:
test=data[0]
test[:,:,0]

array([[158, 158, 158, ..., 152, 152, 152],
       [158, 158, 158, ..., 152, 152, 152],
       [156, 156, 156, ..., 152, 152, 152],
       ..., 
       [ 26,  26,  24, ...,  46,  46,  46],
       [ 26,  26,  25, ...,  45,  47,  43],
       [ 26,  26,  22, ...,  44,  45,  42]], dtype=uint8)

In [15]:
for i in range(len(data)):
    tmp_image=data[i]
    data_out[i,:,:,0] = (tmp_image[:,:,0] - 103.94) * 0.017
    data_out[i,:,:,1] = (tmp_image[:,:,1] - 116.78) * 0.017
    data_out[i,:,:,2] = (tmp_image[:,:,2] - 123.68) * 0.017

In [16]:
# load data then train data
# data,label=load_data()
print(len(label))
label_out=np.zeros((len(label),nb_class))
for i in range(len(label)):
    t=0
    for j in range(len(label[i])):
        tmp=label[i][j]
        tmp=np.array(tmp)
        tmp=np_utils.to_categorical(tmp,nb_class)
        t=t+tmp
        label_out[i]=t
# data_out = np.array(data, dtype="float32")
model=load_new_model()
train_data(data_out,label_out,model)

10665


/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), use_bias=False, strides=(2, 2), name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

9598
Train on 9598 samples, validate on 1067 samples
Epoch 1/15
9598/9598 [==============================] - 435s - loss: 2.0108 - acc: 0.4222 - val_loss: 1.6775 - val_acc: 0.4977
Epoch 2/15
9598/9598 [==============================] - 369s - loss: 1.6574 - acc: 0.4958 - val_loss: 1.6690 - val_acc: 0.4986
Epoch 3/15
9598/9598 [==============================] - 369s - loss: 1.4456 - acc: 0.5598 - val_loss: 1.6763 - val_acc: 0.5117
Epoch 4/15
9598/9598 [==============================] - 369s - loss: 1.2551 - acc: 0.6100 - val_loss: 1.8694 - val_acc: 0.4770
Epoch 5/15
9598/9598 [==============================] - 369s - loss: 1.0739 - acc: 0.6710 - val_loss: 1.8960 - val_acc: 0.4845
Epoch 6/15
9598/9598 [==============================] - 369s - loss: 0.9008 - acc: 0.7270 - val_loss: 2.5720 - val_acc: 0.3852
Epoch 7/15
9598/9598 [==============================] - 369s - loss: 0.7635 - acc: 0.7729 - val_loss: 2.1374 - val_acc: 0.4499
Epoch 8/15
9598/9598 [==============================] - 37

In [17]:

#predict data
out=predict_data()

/root/workspace/densenet161.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), use_bias=False, strides=(2, 2), name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/root/workspace/densenet161.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
/root/workspace/densenet161.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
/root/workspace/densenet161.py:165: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'

In [18]:
p_path="data/test"
image_paths = sorted(list(paths.list_images(p_path)))
image_name=[]
for im_path in image_paths:
    image_name.append(im_path.split('/')[2])
predict_label = pd.read_csv("data/submit_example.csv")
for i in range(predict_label.values.shape[0]):
    find_name=predict_label.values[i][0]
    for j in range(len(image_name)):
        if find_name==image_name[j]:
            tmp_out=out[j]
            list_t=[]
            for m in range(tmp_out.shape[0]):
                if tmp_out[m]>0.2:
                    list_t.append(m+1)
            if len(list_t)==0:
                print(image_name[j])
                list_t.append(np.argmax(tmp_out))
            str_s=""
            for n in range(len(list_t)):
                str_s=str_s+str(list_t[n])
                if n!=len(list_t)-1:
                    str_s=str_s+";"
            predict_label.iloc[i,1]=str_s
predict_label.to_csv("data/predict.csv",index=False)

60a2389ef56c4aa38db15018a458f0b0.jpg
64f925dbc6584bc6a295d53ed2ec5069.jpg
6a5e456e21e0405c8236aae75c23ad7d.jpg
74f952f30d274c91b72842b972d6dfc6.jpg
84edcec63aa4430c8187cdbe531d5d39.png
d3719ed448224695b0c4f03cdaeaa175.jpg
d57de67fb37346899bf5abf01c844bef.jpg
e0409c12e2fd499c8d7ac72aa0b054b9.jpg
e96f89f393ff4179a5048afc4b787e1a.jpg
